Load Julia packages (libraries) needed  for the snippets in chapter 0

In [1]:
using StatisticalRethinking
using CmdStan, StanMCMCChain
gr(size=(500,500));

CmdStan uses a tmp directory to store the output of cmdstan

In [2]:
ProjDir = rel_path("..", "scripts", "04")
cd(ProjDir)

### snippet 4.7

In [3]:
howell1 = CSV.read(rel_path("..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

Use only adults

In [4]:
df2 = filter(row -> row[:age] >= 18, df)
mean_weight = mean(df2[:weight])
df2[:weight_c] = convert(Vector{Float64}, df2[:weight]) .- mean_weight ;

Define the Stan language model

In [5]:
weightsmodel = "
data {
 int < lower = 1 > N; // Sample size
 vector[N] height; // Predictor
 vector[N] weight; // Outcome
}

parameters {
 real alpha; // Intercept
 real beta; // Slope (regression coefficients)
 real < lower = 0 > sigma; // Error SD
}

model {
 height ~ normal(alpha + weight * beta , sigma);
}

generated quantities {
}
";

Define the Stanmodel and set the output format to :mcmcchain.

In [6]:
stanmodel = Stanmodel(name="weights", monitors = ["alpha", "beta", "sigma"],model=weightsmodel,
  output_format=:mcmcchain);
# Input data for cmdstan
heightsdata = Dict("N" => length(df2[:height]), "height" => df2[:height], "weight" => df2[:weight_c]);

Sample using cmdstan

In [7]:
rc, chn, cnames = stan(stanmodel, heightsdata, ProjDir, diagnostics=false,
  summary=false, CmdStanDir=CMDSTAN_HOME);
# Describe the draws
describe(chn)

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
          Mean         SD        Naive SE        MCSE      ESS
alpha 154.5930595 0.273465412 0.00432386782 0.00411035221 1000
 beta   0.9042383 0.042727744 0.00067558495 0.00062675714 1000
sigma   5.1065665 0.190995550 0.00301990481 0.00291444839 1000

Quantiles:
          2.5%        25.0%       50.0%       75.0%        97.5%   
alpha 154.0470000 154.41300000 154.5940000 154.7730000 155.13200000
 beta   0.8231327   0.87473625   0.9041325   0.9337215   0.98981962
sigma   4.7525113   4.97263750   5.1017450   5.2322475   5.48776375



Save the chains in a JLS file

In [8]:
serialize("m4.4s.jls", chn)

chn2 = deserialize("m4.4s.jls")

Object of type "Chains{Float64}"

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Union{Missing, Float64}[154.467 0.946962 5.07883; 154.537 0.946753 5.18517; … ; 154.989 0.918534 4.85561; 154.639 0.887201 5.05504]

Union{Missing, Float64}[155.036 0.888035 5.17977; 154.284 0.904426 5.01136; … ; 154.653 0.969419 5.06085; 153.956 0.934947 4.99526]

Union{Missing, Float64}[154.541 0.84569 4.9717; 154.443 0.833596 5.02684; … ; 154.327 0.934421 5.22032; 154.735 0.864225 5.15099]

Union{Missing, Float64}[154.431 0.859446 5.27437; 154.58 0.891814 4.93622; … ; 154.251 0.963617 5.14029; 154.54 0.828021 5.2504]

Should be identical to earlier result

In [9]:
describe(chn2)

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
          Mean         SD        Naive SE        MCSE      ESS
alpha 154.5930595 0.273465412 0.00432386782 0.00411035221 1000
 beta   0.9042383 0.042727744 0.00067558495 0.00062675714 1000
sigma   5.1065665 0.190995550 0.00301990481 0.00291444839 1000

Quantiles:
          2.5%        25.0%       50.0%       75.0%        97.5%   
alpha 154.0470000 154.41300000 154.5940000 154.7730000 155.13200000
 beta   0.8231327   0.87473625   0.9041325   0.9337215   0.98981962
sigma   4.7525113   4.97263750   5.1017450   5.2322475   5.48776375



End of `m4.4s.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*